In [120]:
import bqplot
import ytree
import yt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets
from ipywidgets import *
from bqplot import DateScale, LinearScale, Axis, Lines, Scatter, Bars, Hist, Figure
from bqplot.interacts import (
    FastIntervalSelector, IndexSelector, BrushIntervalSelector,
    BrushSelector, MultiSelector, LassoSelector, PanZoom, HandDraw)

In [121]:
a = ytree.load("rockstar_halos/out_0.list")
# fn = a.save_arbor()
# a = ytree.load(fn)

In [122]:
print(a.derived_field_list)
print(a.field_list)

['halo_id', 'desc_id', 'mass', 'virial_mass', 'virial_radius', 'scale_radius', 'velocity_dispersion', 'position_x', 'position_y', 'position_z', 'velocity_x', 'velocity_y', 'velocity_z', 'angular_momentum_x', 'angular_momentum_y', 'angular_momentum_z', 'spin_parameter', 'T_|U|', 'redshift']
['ID', 'DescID', 'Mvir', 'Vmax', 'Vrms', 'Rvir', 'Rs', 'Np', 'X', 'Y', 'Z', 'VX', 'VY', 'VZ', 'JX', 'JY', 'JZ', 'Spin', 'rs_klypin', 'Mvir_all', 'M200b', 'M200c', 'M500c', 'M2500c', 'Xoff', 'Voff', 'spin_bullock', 'b_to_a', 'c_to_a', 'A[x]', 'A[y]', 'A[z]', 'b_to_a(500c)', 'c_to_a(500c)', 'A[x](500c)', 'A[y](500c)', 'A[z](500c)', 'T/|U|', 'M_pe_Behroozi', 'M_pe_Diemer', 'scale_factor', 'uid', 'desc_uid']


In [123]:
df_trees = {}
counter = 0
for tree in a:
    scale = tree["prog", "scale_factor"]
    assert(scale.size == np.unique(scale).size)
    Mvir = tree["prog", "Mvir_all"]
    Vrms = tree["prog", "Vrms"]
    Rvir = tree["prog", "Rvir"]
    Spin = tree["prog", "Spin"]
    Z = tree["prog", "redshift"]
    ti = str(counter)
    d = pd.DataFrame({('Mvir'): Mvir, ('Vrms'): Vrms, ('Rvir'): Rvir,('Spin'): Spin,('redshift'): Z})
    df_trees[ti] = d
    counter += 1

df = pd.concat(df_trees)

In [124]:
df2 = df.unstack(0)

In [125]:
redshift_dict = {}
for row in df2["redshift"].itertuples():
    idx = row[0]
    key = np.round(np.max(row[1]),1)
    redshift_dict[key] = {'Rvir':np.array(df2["Rvir"].loc[idx]), 'Mvir':np.array(df2["Mvir"].loc[idx]),
                          'Spin':np.array(df2["Spin"].loc[idx]),'Vrms':np.array(df2["Vrms"].loc[idx])}

In [137]:
x_scale = bqplot.LinearScale()
y_scale = bqplot.LogScale()
np.warnings.filterwarnings('ignore')

sc_x = LinearScale()
sc_y = LinearScale()
sc_xax = Axis(label=('Vrms'), scale=sc_x, orientation='horizontal')
sc_yax = Axis(label=('Mvir'), scale=sc_y, orientation='vertical')
sc_x2 = LinearScale()
sc_y2 = LinearScale()
sc_xax2 = Axis(label=('Vrms'), scale=sc_x2, orientation='horizontal')
sc_yax2 = Axis(label=('Mvir'), scale=sc_y2, orientation='vertical')

Z = 0.00
chosen_y_axis = redshift_dict[Z]["Mvir"]
x_y_cond = np.ones(len(mvir),dtype=bool)

x_orig = redshift_dict[Z]["Vrms"]
y_orig = redshift_dict[Z]["Mvir"]

scatter = bqplot.Scatter(x = x_orig, y = y_orig, 
                        scales = {'x': sc_x, 'y': sc_y}, colors = ['blue'])
scatter2 = bqplot.Scatter(x = x_orig, y = y_orig,
                        scales = {'x': sc_x2, 'y': sc_y2}, colors = ['red'])

db_scat_brush = HTML(value='[]')

# Y-axis selector widget
def dropdown_callback(change):
    chosen_y_axis = redshift_dict[Z][change['new']]
    scatter2.x = x_orig[x_y_cond]
    scatter2.y = chosen_y_axis[x_y_cond]
    fig_scat_brush2.axes =[Axis(label=('Vrms'), scale=sc_x2, orientation='horizontal'),
                           Axis(label=(change['new']), scale=sc_y2, orientation='vertical')]
    fig_scat_brush2.title = 'Vrms vs. ' + change['new']

y_options = ['Rvir', 'Mvir','Spin']

y_selector = widgets.Dropdown(
    handler=dropdown_callback,
    options=y_options,
    value='Mvir',
    description='Y Axis:',
    )

y_selector.observe(dropdown_callback, names=['value'])

# Graph brush selector widget
def brush_callback(change):    
    highlight = bselect.selected
    db_scat_brush.value = str(highlight)
    key = y_selector.value
    chosen_y_axis = redshift_dict[Z][key]
    if change.new is not None:
        try:
            x_condition = (x_orig>=float(highlight[0][0])) & (x_orig<=float(highlight[1][0]))
            y_condition = (y_orig>=float(highlight[0][1])) & (y_orig<=float(highlight[1][1]))
            x_y_cond = x_condition & y_condition
            scatter2.x = x_orig[x_y_cond]
            scatter2.y = chosen_y_axis[x_y_cond]
            fig_scat_brush2.axes =[Axis(label=('Vrms'), scale=sc_x2, orientation='horizontal'),
                       Axis(label=(key), scale=sc_y2, orientation='vertical')]
        except IndexError:
            ;


# Redshift slider widget
redshift_slider = widgets.FloatSlider(value=0.0,min=0.0,max=3.0,step=.1,description='Redshift',
                                      disabled=False,continuous_update=False,
                                      orientation='horizontal',
                                      readout=True,readout_format='.1f',
                                      layout=Layout(width='100%',position='top'))

def on_value_change(change):
    Z = change['new']
    try:
        x_orig = redshift_dict[Z]["Vrms"]
        y_orig = redshift_dict[Z]["Mvir"]
        scatter.x = x_orig
        scatter.y = y_orig
        key = y_selector.value
        chosen_y_axis = redshift_dict[Z][key]
        scatter2.x = x_orig[x_y_cond]
        scatter2.y = chosen_y_axis[x_y_cond]
        fig_scat_brush2.axes =[Axis(label=('Vrms'), scale=sc_x2, orientation='horizontal'),
                               Axis(label=(key), scale=sc_y2, orientation='vertical')]
    except KeyError:
        ;

redshift_slider.observe(on_value_change, names='value')

bselect = bqplot.interacts.BrushSelector(x_scale=sc_x, y_scale=sc_y, marks=[scatter])

bselect.observe(brush_callback, names=['selected'])

fig_scat_brush1 = Figure(marks=[scatter], axes=[sc_xax, sc_yax], title='Vrms vs. Mvir',
                        interaction=bselect)
fig_scat_brush2 = Figure(marks=[scatter2], axes=[sc_xax2, sc_yax2], title='Vrms vs. Mvir')

left_box = VBox([db_scat_brush, fig_scat_brush1])
right_box = VBox([fig_scat_brush2, y_selector])
graph_box = HBox([left_box, right_box])

VBox([graph_box,redshift_slider])

VBox(children=(HBox(children=(VBox(children=(HTML(value='[]'), Figure(axes=[Axis(label='Vrms', scale=LinearScale()), Axis(label='Mvir', orientation='vertical', scale=LinearScale())], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, interaction=BrushSelector(marks=[Scatter(colors=['blue'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, x=array([  345.07998657,   331.95999146,   552.5300293 ,   439.79998779,
         276.42001343,   548.30999756,   409.41000366,   421.98999023,
         354.76998901,   127.55999756,   447.57000732,            nan,
         618.76000977,   389.83999634,   561.17999268,   181.36999512,
         557.77001953,   431.57000732,   524.70001221,   599.66998291,
         285.19000244,   596.7800293 ,   436.42999268,   423.57000732,
        1232.02001953,   158.00999451,   487.01998901,   580.4699707 ,
         514.70001221,   306.91000366,   579.70001221,   530.40997314,
         226.44000244,   570.2800293 ,   582.45001221,            nan,
         580.08001709,   470.55999756,   516.40002441,   580.33001709,
         298.01998901,   546.59997559,   478.76998901,   204.80000305,
         418.98999023,   564.95001221,            nan,   468.54000854,
         335.67001343,   527.25      ,   428.54998779,   559.07000732,
         451.64001465,   387.11999512,   183.1499939 ,   266.63000488,
         461.54998779,   380.60998535,   495.19000244,            nan,
         523.58001709,   216.83000183,   438.69000244,   489.69000244,
         436.97000122,   312.17001343,            nan,   485.88000488,
        1143.92004395,   497.        ,   430.17001343,   356.08999634,
         291.42999268,   267.29998779,   433.98999023,   534.42999268,
         500.45001221,   159.41000366,   354.10998535,   420.14001465,
         438.70001221,   346.23999023,   434.98999023,   286.82000732,
         306.45001221,   289.55999756,   466.33999634,   364.11999512,
         417.77999878,   377.70999146,   147.30000305,   353.07998657,
         384.08999634,            nan,   412.29998779,            nan,
         401.97000122,   440.47000122,   426.70999146,   432.13000488,
         290.17999268,   382.20999146,   333.82000732,   400.85998535,
         432.8500061 ,   226.        ,            nan,   321.91000366,
         384.67999268,   368.73999023,   280.3999939 ,   365.82000732,
        1179.85998535,   674.36999512,   403.91000366,   233.13000488,
         402.60998535,   314.30999756,   291.01000977,   340.70999146,
         381.83999634,    53.88999939,   308.48999023,   244.3999939 ,
        1045.72998047,   307.72000122,   282.07998657,   419.47000122,
         375.07998657,   421.6499939 ,   107.90000153,   156.88000488,
                  nan,   348.51000977,   275.23999023,   470.45999146,
         191.99000549,   368.1000061 ,   380.76998901,   275.16000366,
         338.52999878,   287.82000732,            nan,   372.8999939 ,
         265.79000854,   294.3999939 ,   210.50999451,   324.29000854,
         365.57000732,   331.80999756,   311.1499939 ,   208.69000244,
         318.        ,   154.58999634,   340.42999268,   219.53999329,
         132.66999817,   998.25      ,   303.76998901,   373.77999878,
         229.25999451,   223.88000488,   344.04998779,            nan,
         235.47000122,   248.38999939,            nan,   176.16999817,
         591.9699707 ,   161.1499939 ,   297.73001099,   371.10998535,
         105.86000061,   276.3500061 ,   226.27000427,   216.61000061,
         161.86999512,   379.1000061 ,   145.57000732,            nan,
         262.86999512,   342.3999939 ,   298.64001465,   252.69999695,
         116.1699